In [3]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import json
from datetime import datetime, timezone

# Carrega variáveis do .env
load_dotenv()

TRELLO_KEY = os.getenv("TRELLO_KEY")
TRELLO_TOKEN = os.getenv("TRELLO_TOKEN")
BOARD_ID = os.getenv("BOARD_ID")

params = {
    "key": TRELLO_KEY,
    "token": TRELLO_TOKEN
}

def trello_id_to_datetime(trello_id):
    """Converte o ID do Trello em data de criação do card."""
    timestamp_hex = trello_id[:8]
    timestamp_int = int(timestamp_hex, 16)
    return datetime.fromtimestamp(timestamp_int, tz=timezone.utc)

try:
    # 1. Mapear campos customizados
    custom_fields_url = f"https://api.trello.com/1/boards/{BOARD_ID}/customFields"
    custom_fields_response = requests.get(custom_fields_url, params=params)
    custom_fields_mapping = {}

    if custom_fields_response.status_code == 200:
        for field in custom_fields_response.json():
            custom_fields_mapping[field["id"]] = field["name"]
    else:
        print("⚠️ Falha ao obter campos customizados do board")

    # 2. Obter listas do board
    lists_url = f"https://api.trello.com/1/boards/{BOARD_ID}/lists"
    lists_response = requests.get(lists_url, params=params)
    lists_response.raise_for_status()

    listas = lists_response.json()
    all_cards = []

    for lista in listas:
        list_id = lista["id"]
        list_name = lista["name"]

        # 3. Buscar cards da lista
        cards_url = f"https://api.trello.com/1/lists/{list_id}/cards"
        cards_params = {
            "key": TRELLO_KEY,
            "token": TRELLO_TOKEN,
            "fields": "id,name,url,due,dueComplete,dateLastActivity,labels",
            "customFieldItems": "true",
        }

        cards_response = requests.get(cards_url, params=cards_params)
        if cards_response.status_code != 200:
            continue

        for card in cards_response.json():
            created_date = trello_id_to_datetime(card["id"])
            completed_date = None

            # Detectar data de conclusão
            if card.get("dueComplete") and card.get("due"):
                completed_date = datetime.fromisoformat(card["due"].replace("Z", "+00:00"))
            else:
                # Verificar se houve fechamento do card
                for action in card.get("actions", []):
                    if action["type"] == "updateCard" and action["data"].get("card", {}).get("closed") is True:
                        completed_date = datetime.fromisoformat(action["date"].replace("Z", "+00:00"))
                        break

            # Calcular ciclo em dias e horas
            cycle_time_days_hours = None
            if completed_date:
                delta = completed_date - created_date
                total_hours = delta.total_seconds() // 3600
                days_part = int(total_hours // 24)
                hours_part = int(total_hours % 24)
                cycle_time_days_hours = f"{days_part}d {hours_part}h"

            card_data = {
                "list_id": list_id,
                "list_name": list_name,
                "card_id": card.get("id"),
                "card_name": card.get("name"),
                "created_date": created_date.isoformat(),
                "completed_date": completed_date.isoformat() if completed_date else None,
                "cycle_time_days_hours": cycle_time_days_hours,
                "card_labels": ", ".join([label["name"] for label in card.get("labels", [])]),
            }

            # Campos customizados
            for cfi in card.get("customFieldItems", []):
                field_id = cfi.get("idCustomField")
                field_name = custom_fields_mapping.get(field_id, f"custom_{field_id}")
                value = cfi.get("value")
                id_value = cfi.get("idValue")
                value_final = None

                if value:
                    value_final = list(value.values())[0]
                elif id_value:
                    value_final = id_value

                card_data[f"custom_{field_name}"] = value_final

            all_cards.append(card_data)

    # 4. Criar DataFrame e salvar CSV
    df = pd.DataFrame(all_cards)
    df.to_csv("storypoints_bugs_ciclo.csv", index=False, encoding="utf-8-sig")

    # 5. Salvar JSON e exibir
    with open("storypoints_bugs_ciclo.json", "w", encoding="utf-8") as json_file:
        json.dump(all_cards, json_file, ensure_ascii=False, indent=2)

    print("✅ Arquivo 'storypoints_bugs_ciclo.csv' gerado com sucesso!")
    print("✅ Arquivo 'storypoints_bugs_ciclo.json' gerado com sucesso!")
    print("\n📄 Conteúdo JSON:")
    print(json.dumps(all_cards, indent=2, ensure_ascii=False))

except requests.exceptions.RequestException as err:
    print(f"❌ Erro na requisição: {err}")
except Exception as e:
    print(f"❌ Erro inesperado: {e}")


✅ Arquivo 'storypoints_bugs_ciclo.csv' gerado com sucesso!
✅ Arquivo 'storypoints_bugs_ciclo.json' gerado com sucesso!

📄 Conteúdo JSON:
[
  {
    "list_id": "6891356a01e4dbb84aeef7f8",
    "list_name": "Concluído",
    "card_id": "689142d753d78083d0e88da8",
    "card_name": "Teste concluido",
    "created_date": "2025-08-04T23:31:35+00:00",
    "completed_date": "2025-08-07T21:00:00+00:00",
    "cycle_time_days_hours": "2d 21h",
    "card_labels": "Bugs",
    "custom_StoryPoint": "Apenas mais um teste de story point para coleta de dados via api"
  },
  {
    "list_id": "6891356a01e4dbb84aeef7f8",
    "list_name": "Concluído",
    "card_id": "689141723404eee63cbbaa45",
    "card_name": "Teste a fazer",
    "created_date": "2025-08-04T23:25:38+00:00",
    "completed_date": "2025-08-05T21:00:00+00:00",
    "cycle_time_days_hours": "0d 21h",
    "card_labels": "Bugs",
    "custom_StoryPoint": "Story Point para exemplo de coleta de dados específicos de um card no board do trello."
  },
  {